In [1]:
#黃歆宇, 112356036, 資管碩一, AS_01

import socket
import threading

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
pars = ('127.0.0.1', 8888) #設定port
s.bind(pars) #連接port
s.listen(5)

def serveClient(clientsocket, address):
    # a loop to continuously receive messages from the client
    while True: 
        # then receive at most 1024 bytes message and store these bytes in a variable named 'data'
        data = clientsocket.recv(1024)

        # 若在網址輸入"http://127.0.0.1:8888/題目要求的html或css"，會發生的事"
        # 狀態200且顯示good.html
        if b"/good.html" in data:
            res_for_good = (
                "HTTP/1.1 200 OK\r\n"
                "Content-Type: text/html\r\n"
                "Connection: Keep-Alive\r\n"
                "\r\n"
                '''<html><head><title>200 OK</title><style>body{color:red;}</style></head><body>My student ID: 112356036</body></html>'''
                #<link href="style.css" rel="stylesheet" type="text/css">
            )
                    
            clientsocket.send(res_for_good.encode())

        # 讓使用者可以連線到負責html樣式設計的.css檔
        # 但一直沒有成功，無法導入格式到 good.html
        elif b"/style.css" in data:
            res_for_style = (
                "HTTP/1.1 200 OK\r\n"
                "Content-Type: text/css\r\n"
                "Connection: Keep-Alive\r\n"
                "\r\n"
                "body {color:red;}"
            )
            clientsocket.send(res_for_style.encode())
            
        # 301顯示且導回good.html
        elif b"/redirect.html" in data:
            response_r = (
                "HTTP/1.1 301 Moved Permanently\r\n"
                "Location: /good.html\r\n"
                "Content-Type: text/html\r\n"
                "Connection: Keep-Alive\r\n"
                "\r\n"
                '''<html><head><title>300 Permanently</title></head></html>'''
            )
            clientsocket.send(response_r.encode())
        
        # 404顯示
        elif b"/notfound.html" in data:
            response_n = (
                "HTTP/1.1 404 Not Found\r\n"
                "Content-Type: text/html\r\n"
                "Connection: Keep-Alive\r\n"
                "\r\n"
                '''<html><head><title>404 Not found</title></head><body>404 notfound</body></html>''' # 測試字樣，確認真的有跳轉到404頁面
                )            
            clientsocket.send(response_n.encode())

        
        # if the client sends some termination message to the server, then the server close the socket
        elif data == b'close' in data:
            clientsocket.close()
            break


while True:
    # accept a new client and get it's information
    (clientsocket, address) = s.accept()
    
    # create a new thread to serve this new client
    # after the thread is created, it will start to execute 'target' function with arguments 'args' 
    threading.Thread(target = serveClient, args = (clientsocket, address)).start()